# 05 CNN-Simple

In [ ]:
import config
from utils import data as utils_data
from utils.model import Model

import pandas as pd
import numpy as np
import tensorflow as tf

## Load data

In [ ]:
data, imgs_left, imgs_right = utils_data.load(
    config.PATH_DATA, config.PATH_IMGS_LEFT, config.PATH_IMGS_RIGHT
)

In [ ]:
data.head()

## Split data

In [ ]:
(
    (train_data, train_imgs_left, train_imgs_right),
    (validation_data, validation_imgs_left, validation_imgs_right),
    (test_data, test_imgs_left, test_imgs_right)
) = utils_data.split(
    data, imgs_left, imgs_right,
    train_size=0.95,
    validation_size=0.95,
    random_state=42
)

In [ ]:
print("Train length: {}".format(len(train_data)))
print("Validation length: {}".format(len(validation_data)))
print("Test length: {}".format(len(test_data)))

## Model

### Architecture

In [ ]:
def get_model(features, left_imgs, right_imgs, keep_prob):
    new_shape = np.array([-1, 20, 30, 1])
    with tf.variable_scope('model'):
        # Left Eye Img
        l_input = tf.reshape(left_imgs, new_shape)
        # 20x30x1
        cnn_l_01 = tf.layers.conv2d(
            inputs=l_input, 
            filters=32, 
            kernel_size=3, 
            strides=2,
            padding="SAME"
        )
        # 10x15x32
        cnn_l_02 = tf.layers.conv2d(
            inputs=cnn_l_01, 
            filters=64, 
            kernel_size=3, 
            strides=2,
            padding="VALID"
        ) 
        # 4x7x64
        
        # Right Eye Img  
        r_input = tf.reshape(right_imgs, new_shape)      
        # 20x30x1
        cnn_r_01 = tf.layers.conv2d(
            inputs=r_input, 
            filters=32, 
            kernel_size=3, 
            strides=2,
            padding="SAME"
        )
        # 10x15x32
        cnn_r_02 = tf.layers.conv2d(
            inputs=cnn_r_01, 
            filters=64, 
            kernel_size=3, 
            strides=2,
            padding="VALID"
        )
        # 4x7x64
        
        # Flatten convs, concat & dense        
        left_flat = tf.contrib.layers.flatten (cnn_l_02)
        right_flat =  tf.contrib.layers.flatten (cnn_r_02)
        img_concat = tf.concat(
            values=[left_flat, right_flat],
            axis=1
        )
        img_dense = tf.layers.dense(
            inputs=img_concat,
            units=256,
            activation=tf.nn.relu,
        )
        
        # Concat imgs with features, dense x 2 and output
        global_concat = tf.concat(
            values=[features, img_dense],
            axis=1
        )
        global_dense01 = tf.layers.dense(global_concat, 128, activation=tf.nn.relu)
        global_dropout01 = tf.nn.dropout(
            x=global_dense01,
            keep_prob=keep_prob
        )
        global_dense02 = tf.layers.dense(global_dropout01, 64, activation=tf.nn.relu)
        global_dense03 = tf.layers.dense(global_dense02, 2, activation=None)
        
        return global_dense03

### Parameters

In [ ]:
MODEL_NAME = 'cnn_simple-06'

EPOCHS = 100
BATCH_SIZE = 512
LEARNING_RATE = 0.0002
KEEP_PROB = 0.85

## Runs

In [ ]:
model = Model(MODEL_NAME, get_model)

### Train

In [ ]:
model.train(
    train_data, train_imgs_left, train_imgs_right,
    validation_data, validation_imgs_left, validation_imgs_right,
    BATCH_SIZE, EPOCHS, LEARNING_RATE, KEEP_PROB
)

### Test

In [ ]:
model_test = Model(MODEL_NAME, saved_model=MODEL_NAME+".final")

In [ ]:
model_test.test(
    test_data, test_imgs_left, test_imgs_right
)

### Results


| Name | Epochs | Batch Size | Learning rate  | Keep_prob | Train | Validation | Test |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| cnn_simple-01 | 10 | 256 | 0.003 | 1.0 | 0.13434778153896332 | 0.19420433044433594 | 0.19306178 |
| cnn_simple-02 | 40 | 256 | 0.0005 | 0.9 | 0.06496571749448776 | 0.16198869049549103 | 0.15953958 |
| cnn_simple-03 | 40 | 256 | 0.0001 | 0.8 | 0.10085277259349823 | 0.17251187562942505 | 0.16782267 |
| cnn_simple-04 | 50 | 256 | 0.0003 | 0.85 | 0.06626483052968979 | 0.16012543439865112 | 0.16782267 |
| cnn_simple-05 | 100 | 256 | 0.001 | 0.50 | 0.09738671779632568 | 0.19700293242931366 | 0.1929463 |
| cnn_simple-06 | 50 | 256 | 0.0001 | 0.7 | 0.09397947788238525 | 0.17506185173988342 | 0.17135754 |
| cnn_simple-07 | 100 | 256 | 0.0001 | 0.7 | 0.07396189123392105 | 0.16842615604400635 | 0.16878742 |
| cnn_simple-08 | 100 | 512 | 0.0001 | 0.9 | 0.07066092640161514 | 0.16528385877609253 | 0.16040522 |
| cnn_simple-09 | 100 | 512 | 0.0002 | 0.85 | 0.058131735771894455 | 0.16360823810100555 | 0.16219755 |